# Version 1 - Populate Pinecone with movie poster embeddings and metadata

V1 movie poster dataset sourced from Hugging Face:
https://huggingface.co/datasets/pinecone/movie-posters

V2 will source movies using TMDB API's popular endpoint, to get a list of more modern movies (and movie posters of better image quality)
https://developer.themoviedb.org/reference/movie-popular-list

In [10]:
!pip install datasets
!pip install pinecone-client
!pip install tmdbv3api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 require

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.6/117.6 kB 10.3 MB/s eta 0:00:00


CLIP

In [ ]:
from datasets import load_dataset
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
from io import BytesIO
import pinecone
from tmdbv3api import TMDb, Find
from google.colab import userdata
import time
from pinecone import Pinecone
from pinecone import ServerlessSpec
from google.colab import userdata

# Configuration Variables
# MODEL_NAME = "openai/clip-vit-base-patch32"
# PROCESSOR_NAME = "openai/clip-vit-base-patch32"
# NAMESPACE = "CLIP"

MODEL_NAME = "openai/clip-vit-base-patch32"
PROCESSOR_NAME = "openai/clip-vit-base-patch32"
NAMESPACE = "CLIP"

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get('PINECONE_KEY'))
index = pc.Index("movie-posters")

# Initialize TMDB API client
tmdb = TMDb()
tmdb.api_key = userdata.get("TMDB_API_KEY")

# Initialize the Find class
find = Find()

# Load the Pinecone movie poster dataset
ds = load_dataset("pinecone/movie-posters")

# Initialize CLIP model and processor
model = CLIPModel.from_pretrained(MODEL_NAME)
processor = CLIPProcessor.from_pretrained(PROCESSOR_NAME)

# Function to fetch an image from a URL and get its embedding
def get_image_embedding(poster_url):
    response = requests.get(poster_url)
    image = Image.open(BytesIO(response.content))
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        embedding = model.get_image_features(**inputs)
    return embedding.squeeze().numpy()

# Function to fetch movie metadata from TMDB API using IMDb ID
def get_movie_metadata(imdb_id):
    time.sleep(0.02)  # 50 requests per second = 1 request every 0.02 seconds so we don't overload TMDB API
    try:
        # Use the Find class to search for the movie by IMDb ID
        results = find.find(imdb_id, 'imdb_id')
        if results.movie_results:
            movie = results.movie_results[0]  # Take the first result
            return {
                'title': movie.title,
                'overview': movie.overview,
                'release_date': movie.release_date
            }
        else:
            print(f"No metadata found for IMDb ID {imdb_id}")
            return None
    except Exception as e:
        print(f"Error fetching metadata for IMDb ID {imdb_id}: {e}")
        return None

# Process the dataset and upload embeddings with metadata to Pinecone
def process_and_upload():
    for item in ds['train']:
        imdb_id = item['imdbId']
        poster_url = item['poster']

        # Fetch movie metadata
        metadata = get_movie_metadata(imdb_id)
        if metadata is None:
            continue

        # Add poster URL from dataset to metadata
        metadata['poster_url'] = poster_url

        # Generate image embedding
        try:
            embedding = get_image_embedding(poster_url)
        except Exception as e:
            print(f"Failed to get embedding for {poster_url}: {e}")
            continue

        # Upload to Pinecone with namespace and metadata as a dictionary
        index.upsert(
            vectors=[(imdb_id, embedding, metadata)],
            namespace=NAMESPACE
        )

#process_and_upload()

Generating train split:   0%|          | 0/10269 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
get_movie_metadata("tt6087562")

{'title': 'The Scythian Lamb',
 'overview': "Based on a manga written by Tatsuhiko Yamagami, the story is set in a former seaport town Uobuka, where 6 former criminals were sent to live there by the government, with the intention of re-socialising them. Aside from the few who know about the project, the general townsfolk has no idea of the former convicts' identities. Tsukisue is the pleasant and efficient municipal official put in charge of the programme. As he slowly learns about their past, a body is discovered.",
 'release_date': '2018-02-03'}

In [ ]:
# Pinecone and create an index
from pinecone import Pinecone
from pinecone import ServerlessSpec
from google.colab import userdata

pc = Pinecone(api_key=userdata.get('PINECONE_KEY'))

# pc.create_index(
#     name="movie-posters",
#     dimension=512, # dimensions from CLIP
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

# Access the existing index
index = pc.Index("movie-posters")


In [ ]:
#index.delete(delete_all=True, namespace='')

{}

In [ ]:
print(index.describe_index_stats())

{'dimension': 512,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10191}},
 'total_vector_count': 10191}


ResNet

In [ ]:
from datasets import load_dataset
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import pinecone
from tmdbv3api import TMDb, Find
from google.colab import userdata
import time

# Configuration Variables
MODEL_NAME = "resnet50"  # Using ResNet50 model
NAMESPACE = "ResNet-50"
VECTOR_DIM = 512  # Limiting to 512 to match CLIP dimensions (Pinecone free plan has only one index)

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get('PINECONE_KEY'))
index = pc.Index("movie-posters")

# Initialize TMDB API client
tmdb = TMDb()
tmdb.api_key = userdata.get("TMDB_API_KEY")

# Initialize the Find class
find = Find()

# Load the Pinecone movie poster dataset
ds = load_dataset("pinecone/movie-posters")

# Initialize ResNet model and modify it to produce 512-dimensional embeddings
class ResNet50WithEmbedding(torch.nn.Module):
    def __init__(self, embedding_dim):
        super(ResNet50WithEmbedding, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        # Remove the final fully connected layer
        self.base_model = torch.nn.Sequential(*list(self.base_model.children())[:-1])
        # Add a new fully connected layer
        self.fc = torch.nn.Linear(in_features=2048, out_features=embedding_dim)

    def forward(self, x):
        x = self.base_model(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x

model = ResNet50WithEmbedding(VECTOR_DIM)
model.eval()

# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to fetch an image from a URL and get its embedding
def get_image_embedding(poster_url):
    response = requests.get(poster_url)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    inputs = preprocess(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        embedding = model(inputs).squeeze().numpy()
    return embedding

# Function to fetch movie metadata from TMDB API using IMDb ID
def get_movie_metadata(imdb_id):
    time.sleep(0.02)  # 50 requests per second = 1 request every 0.02 seconds so we don't overload TMDB API
    try:
        results = find.find(imdb_id, 'imdb_id')
        if results.movie_results:
            movie = results.movie_results[0]  # Take the first result
            return {
                'title': movie.title,
                'overview': movie.overview,
                'release_date': movie.release_date
            }
        else:
            print(f"No metadata found for IMDb ID {imdb_id}")
            return None
    except Exception as e:
        print(f"Error fetching metadata for IMDb ID {imdb_id}: {e}")
        return None

# Process the dataset and upload embeddings with metadata to Pinecone
def process_and_upload():
    for item in ds['train']:
        imdb_id = item['imdbId']
        poster_url = item['poster']

        # Fetch movie metadata
        metadata = get_movie_metadata(imdb_id)
        if metadata is None:
            continue

        # Add poster URL from dataset to metadata
        metadata['poster_url'] = poster_url

        # Generate image embedding
        try:
            embedding = get_image_embedding(poster_url)
        except Exception as e:
            print(f"Failed to get embedding for {poster_url}: {e}")
            continue

        # Upload to Pinecone with the specified namespace and metadata as a dictionary
        index.upsert(
            vectors=[(imdb_id, embedding.tolist(), metadata)],
            namespace=NAMESPACE
        )

#process_and_upload()

Failed to get embedding for https://m.media-amazon.com/images/M/MV5BMDkzNmRhNTMtZDI4NC00Zjg1LTgxM2QtMjYxZDQ3OWJlMDRlXkEyXkFqcGdeQXVyNTU5MjkzMTU@._V1_SX300.jpg: cannot identify image file <_io.BytesIO object at 0x7fe0b7856ed0>
No metadata found for IMDb ID tt5491994
No metadata found for IMDb ID tt3218680
No metadata found for IMDb ID tt3012698
Failed to get embedding for https://m.media-amazon.com/images/M/MV5BYzE2MjEwMTQtOTQ2Mi00ZWExLTkyMjUtNmJjMjBlYWFjZDdlXkEyXkFqcGdeQXVyMTI3ODAyMzE2._V1_SX300.jpg: cannot identify image file <_io.BytesIO object at 0x7fe0b72de1b0>
No metadata found for IMDb ID tt2879552
No metadata found for IMDb ID tt2401256
No metadata found for IMDb ID tt6769208
Failed to get embedding for N/A: Invalid URL 'N/A': No scheme supplied. Perhaps you meant https://N/A?
No metadata found for IMDb ID tt4299972
No metadata found for IMDb ID tt5189670
No metadata found for IMDb ID tt7343762
No metadata found for IMDb ID tt3762944
Failed to get embedding for https://m.media-a

VGG16

In [ ]:
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import pinecone
from tmdbv3api import TMDb, Find
from google.colab import userdata
import time

# Configuration Variables
VECTOR_DIM = 512  # Dimension for Pinecone index, limited to 512 by CLIP, and only one index allowed on free tier Pinecone
NAMESPACE = "VGG16"

# Initialize Pinecone
pc = Pinecone(api_key=userdata.get('PINECONE_KEY'))
index = pc.Index("movie-posters")

# Initialize TMDB API client
tmdb = TMDb()
tmdb.api_key = userdata.get("TMDB_API_KEY")

# Initialize the Find class
find = Find()

# Load the Pinecone movie poster dataset
ds = load_dataset("pinecone/movie-posters")

# Initialize VGG16 model and modify it to produce 512-dimensional embeddings
class VGG16WithEmbedding(torch.nn.Module):
    def __init__(self, embedding_dim):
        super(VGG16WithEmbedding, self).__init__()
        self.base_model = models.vgg16(pretrained=True)
        # Remove the final fully connected layers
        self.base_model.classifier = torch.nn.Sequential(*list(self.base_model.classifier.children())[:-3])
        # Add a new fully connected layer
        self.fc = torch.nn.Linear(in_features=4096, out_features=embedding_dim)

    def forward(self, x):
        x = self.base_model(x)
        x = self.fc(x)
        return x

model = VGG16WithEmbedding(VECTOR_DIM)
model.eval()

# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to fetch an image from a URL and get its embedding
def get_image_embedding(poster_url):
    response = requests.get(poster_url)
    image = Image.open(BytesIO(response.content)).convert('RGB')
    inputs = preprocess(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        embedding = model(inputs).squeeze().numpy()
    return embedding

# Function to fetch movie metadata from TMDB API using IMDb ID
def get_movie_metadata(imdb_id):
    time.sleep(0.02)  # 50 requests per second = 1 request every 0.02 seconds so we don't overload TMDB API
    try:
        results = find.find(imdb_id, 'imdb_id')
        if results.movie_results:
            movie = results.movie_results[0]  # Take the first result
            return {
                'title': movie.title,
                'overview': movie.overview,
                'release_date': movie.release_date
            }
        else:
            print(f"No metadata found for IMDb ID {imdb_id}")
            return None
    except Exception as e:
        print(f"Error fetching metadata for IMDb ID {imdb_id}: {e}")
        return None

# Process the dataset and upload embeddings with metadata to Pinecone
def process_and_upload():
    for item in ds['train']:
        imdb_id = item['imdbId']
        poster_url = item['poster']

        # Fetch movie metadata
        metadata = get_movie_metadata(imdb_id)
        if metadata is None:
            continue

        # Add poster URL from dataset to metadata
        metadata['poster_url'] = poster_url

        # Generate image embedding
        try:
            embedding = get_image_embedding(poster_url)
        except Exception as e:
            print(f"Failed to get embedding for {poster_url}: {e}")
            continue

        # Upload to Pinecone with the specified namespace and metadata as a dictionary
        index.upsert(
            vectors=[(imdb_id, embedding.tolist(), metadata)],
            namespace=NAMESPACE
        )

#process_and_upload()


Failed to get embedding for https://m.media-amazon.com/images/M/MV5BMDkzNmRhNTMtZDI4NC00Zjg1LTgxM2QtMjYxZDQ3OWJlMDRlXkEyXkFqcGdeQXVyNTU5MjkzMTU@._V1_SX300.jpg: cannot identify image file <_io.BytesIO object at 0x7fe075541cb0>
No metadata found for IMDb ID tt5491994
No metadata found for IMDb ID tt3218680
No metadata found for IMDb ID tt3012698
Failed to get embedding for https://m.media-amazon.com/images/M/MV5BYzE2MjEwMTQtOTQ2Mi00ZWExLTkyMjUtNmJjMjBlYWFjZDdlXkEyXkFqcGdeQXVyMTI3ODAyMzE2._V1_SX300.jpg: cannot identify image file <_io.BytesIO object at 0x7fe075541c60>
No metadata found for IMDb ID tt2879552
No metadata found for IMDb ID tt2401256
No metadata found for IMDb ID tt6769208
Failed to get embedding for N/A: Invalid URL 'N/A': No scheme supplied. Perhaps you meant https://N/A?
No metadata found for IMDb ID tt4299972
No metadata found for IMDb ID tt5189670
No metadata found for IMDb ID tt7343762
No metadata found for IMDb ID tt3762944
Failed to get embedding for https://m.media-a

# Mapping from imdb id to movie name
##(for the user to be able to query on only movie names which exist in the dataset)

In [ ]:
# pc = Pinecone(api_key=userdata.get('PINECONE_KEY'))
# index = pc.Index("movie-posters")
# namespaces =['ResNet-50', 'VGG16', 'CLIP']

# mapping = {}
# for ns in namespaces:
#   for ids in index.list(namespace=ns):
#     #print(ids)
#     res = index.fetch(ids=ids, namespace=ns)

#     vectors = res['vectors']
#     for k in vectors:
#       title = vectors[k]['metadata']['title']
#       if title not in mapping:
#         mapping[title] = k
# print(mapping)

{'You Can’t Watch This': 'tt10322936', 'Picture a Perfect Christmas': 'tt10553910', 'Wolf Creek 2': 'tt2004432', 'The Centrifuge Brain Project': 'tt2005369', 'Uncanny Annie': 'tt10925782', 'Back to School': 'tt10715194', 'The Black Godfather': 'tt10289996', 'Warriors of the Rainbow: Seediq Bale - Part 1: The Sun Flag': 'tt2007993', 'Backwards': 'tt2005153', 'Seth Meyers: Lobby Baby': 'tt11168100', "King's Faith": 'tt2004271', 'Billion': 'tt10214842', 'Les Misérables': 'tt10199590', 'Breathless': 'tt2005164', 'Love, Fall & Order': 'tt10887504', 'Machete Kills': 'tt2002718', 'Radio Rebel': 'tt2008633', 'Return to Horror Hotel': 'tt10581102', 'The Adopted': 'tt2004279', 'Arsenio Hall: Smart and Classy': 'tt10150694', 'Dave Chappelle: Sticks & Stones': 'tt10810424', 'Jellyfish Eyes': 'tt2006181', 'Sturgill Simpson Presents Sound & Fury': 'tt10682000', 'Roger Waters: Us + Them': 'tt10121372', 'Zombies': 'tt10514516', 'First Position': 'tt2008513', 'Anthony Jeselnik: Fire in the Maternity Wa

In [ ]:
# print(len(mapping))

9863


# Test embedding results

In [ ]:
import random

# Create a dictionary for quick lookup of poster URLs by IMDb ID
id_to_poster = {item['imdbId']: item['poster'] for item in ds['train']}

# Choose a random movie from the dataset
random_movie = random.choice(ds['train'])

# Extract the IMDb ID and poster URL of the random movie
specific_imdb_id = random_movie['imdbId']

id_to_poster = {item['imdbId']: item['poster'] for item in ds['train']}

# Look up the movie in the dataset using the specific IMDb ID
if specific_imdb_id in id_to_poster:
    poster_url = id_to_poster[specific_imdb_id]

    # Get the embedding of the specified movie (use the function get_image_embedding)
    embedding = get_image_embedding(poster_url)  # Assuming you have already defined this function

    # Query Pinecone to get the top 10 closest movie posters
    result = index.query(vector=embedding.tolist(), top_k=10, include_metadata=True, namespace="CLIP")

    # Print the specified movie and the top 10 similar movies with metadata
    print(f"Specified Movie IMDb ID: {specific_imdb_id}")
    print(f"Poster URL: {poster_url}")
    print("Top 10 Similar Movies:")
    for match in result['matches']:
        similar_imdb_id = match['id']
        similar_poster_url = id_to_poster.get(similar_imdb_id, "URL not found")
        # Access the metadata
        metadata = match.get('metadata', {})
        title = metadata.get('title', "Title not found")
        overview = metadata.get('overview', "Overview not found")
        release_date = metadata.get('release_date', "Release date not found")
        poster_url = metadata.get('poster_url', "Poster URL not found")

        print(f"IMDb ID: {similar_imdb_id}, Score: {match['score']}")
        print(f"Title: {title}")
        print(f"Overview: {overview}")
        print(f"Release Date: {release_date}")
        print(f"Poster URL: {poster_url}")
else:
    print(f"IMDb ID {specific_imdb_id} not found in the dataset.")


Specified Movie IMDb ID: tt8523678
Poster URL: https://m.media-amazon.com/images/M/MV5BZDIwMWNmNzctNjFlYS00ZTMyLTlmOWEtYThiZjJjYjcwNmE5XkEyXkFqcGdeQXVyMjk3NTUyOTc@._V1_SX300.jpg
Top 10 Similar Movies:
IMDb ID: tt8523678, Score: 1.00005567
Title: Flesh & Blood
Overview: Kimberly, a teenager suffering from agoraphobia, has not left the house since her mother's unsolved murder. On the eve of Thanksgiving, she begins to suspect that the safe harbor of home and her doting father may be a dangerous mirage.
Release Date: 2018-11-02
Poster URL: https://m.media-amazon.com/images/M/MV5BZDIwMWNmNzctNjFlYS00ZTMyLTlmOWEtYThiZjJjYjcwNmE5XkEyXkFqcGdeQXVyMjk3NTUyOTc@._V1_SX300.jpg
IMDb ID: tt4670016, Score: 0.730577171
Title: Wolves at the Door
Overview: Four friends gather at an elegant home during the Summer of Love, 1969. Unbeknownst to them, deadly visitors are waiting outside. What begins as a simple farewell party turns to a night of primal terror as the intruders stalk and torment the four, who

In [ ]:
# Specify the IMDb ID of the movie you want to use
specific_imdb_id = 'tt3638686'

id_to_poster = {item['imdbId']: item['poster'] for item in ds['train']}

# Look up the movie in the dataset using the specific IMDb ID
if specific_imdb_id in id_to_poster:
    poster_url = id_to_poster[specific_imdb_id]

    # Get the embedding of the specified movie (use the function get_image_embedding)
    embedding = get_image_embedding(poster_url)  # Assuming you have already defined this function

    # Query Pinecone to get the top 10 closest movie posters
    result = index.query(vector=embedding.tolist(), top_k=10, include_metadata=True, namespace="CLIP")

    # Print the specified movie and the top 10 similar movies with metadata
    print(f"Specified Movie IMDb ID: {specific_imdb_id}")
    print(f"Poster URL: {poster_url}")
    print("Top 10 Similar Movies:")
    for match in result['matches']:
        similar_imdb_id = match['id']
        similar_poster_url = id_to_poster.get(similar_imdb_id, "URL not found")
        # Access the metadata
        metadata = match.get('metadata', {})
        title = metadata.get('title', "Title not found")
        overview = metadata.get('overview', "Overview not found")
        release_date = metadata.get('release_date', "Release date not found")
        poster_url = metadata.get('poster_url', "Poster URL not found")

        print(f"IMDb ID: {similar_imdb_id}, Score: {match['score']}")
        print(f"Title: {title}")
        print(f"Overview: {overview}")
        print(f"Release Date: {release_date}")
        print(f"Poster URL: {poster_url}")
else:
    print(f"IMDb ID {specific_imdb_id} not found in the dataset.")


Specified Movie IMDb ID: tt3638686
Poster URL: https://m.media-amazon.com/images/M/MV5BNzI0MTk5ODg4MF5BMl5BanBnXkFtZTgwODQ2NjgwOTE@._V1_SX300.jpg
Top 10 Similar Movies:
IMDb ID: tt3638686, Score: 1.00113225
Title: Famous Nathan
Overview: A Coney Island-inspired, densely-layered visually dynamic documentary portrait of the life and times of the original Nathan's Famous, created in 1916 by filmmaker Lloyd Handwerker's grandparents, Nathan and Ida Handwerker. 30 years in the making, Famous Nathan interweaves decades-spanning archival footage, family photos and home movies, an eclectic soundtrack and never-before-heard audio from Nathan: his only interview, ever as well as compelling, intimate and hilarious interviews with the dedicated band of workers, not at all shy at offering opinions, memories and the occasional tall tale.
Release Date: 2015-07-17
Poster URL: https://m.media-amazon.com/images/M/MV5BNzI0MTk5ODg4MF5BMl5BanBnXkFtZTgwODQ2NjgwOTE@._V1_SX300.jpg
IMDb ID: tt4191702, Score: 0

Test TMDB API

In [ ]:
import requests

tmdb_api_key = userdata.get('TMDB_API_KEY')

# Function to get movie details from TMDB using IMDb ID
def get_movie_details(imdb_id):
    url = f"https://api.themoviedb.org/3/find/{imdb_id}"
    params = {
        'api_key': tmdb_api_key,
        'external_source': 'imdb_id'
    }
    response = requests.get(url, params=params)
    data = response.json()

    # print("respones data")
    # print(data)

    if response.status_code == 200 and 'movie_results' in data:
        if len(data['movie_results']) > 0:
            return data['movie_results'][0]  # Return the first movie result
        else:
            print("No movie found for this IMDb ID.")
            return None
    else:
        print(f"Error: {response.status_code}")
        return None

# Example usage:
imdb_id = 'tt2514894'  # Example IMDb ID
movie_details = get_movie_details(imdb_id)

print(movie_details)

if movie_details:
    print("Title:", movie_details.get('title'))
    print("Overview:", movie_details.get('overview'))
    print("Release Date:", movie_details.get('release_date'))
    print("Poster Path:", id_to_poster.get(imdb_id, "URL not found"))


{'backdrop_path': None, 'id': 173465, 'title': 'Medora', 'original_title': 'Medora', 'overview': "In America's basketball heartland, four resilient boys from rural Medora, Indiana, fight to end their high school team's three-year losing streak, as their dwindling town faces the threat of extinction.", 'poster_path': None, 'media_type': 'movie', 'adult': False, 'original_language': 'en', 'genre_ids': [99], 'popularity': 0.514, 'release_date': '2013-11-08', 'video': False, 'vote_average': 7.2, 'vote_count': 4}
Title: Medora
Overview: In America's basketball heartland, four resilient boys from rural Medora, Indiana, fight to end their high school team's three-year losing streak, as their dwindling town faces the threat of extinction.
Release Date: 2013-11-08
Poster Path: https://m.media-amazon.com/images/M/MV5BMTU0MzM2ODI2N15BMl5BanBnXkFtZTgwNTE0NDc2MDE@._V1_SX300.jpg


# Version 2 - Populate separate indexes using "popular" TMDB endpoint

In [9]:
# Pinecone and create an index
from pinecone import Pinecone
from pinecone import ServerlessSpec
from google.colab import userdata

pc = Pinecone(api_key=userdata.get('PINECONE_KEY'))

pc.create_index(
    name="movie-posters-v2-clip",
    dimension=768, # dimensions from CLIP
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

# pc.create_index(
#     name="movie-posters-v2-resnet-50",
#     dimension=2048,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

# pc.create_index(
#     name="movie-posters-v2-vgg16",
#     dimension=4096,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

# Access the existing index
#index = pc.Index("movie-posters")

Getting top 10,000 popular movies (limited by page 500 max for tmdb, 20 results per page)

In [1]:
import requests
from google.colab import userdata
import json
import pprint
import time

def get_popular_page(page, TMDB_AUTH):
  '''
  Returns results page as python dict
  20 movie results per page
  '''
  time.sleep(0.02)  # 50 requests per second = 1 request every 0.02 seconds so we don't overload TMDB API
  url = f"https://api.themoviedb.org/3/movie/popular?language=en-US&page={page}"
  headers = {
    "accept": "application/json",
    "Authorization": TMDB_AUTH
  }
  try:
    response = requests.get(url, headers=headers)
  except Exception as e:
    print(f"Problem getting response for page {page} {e}")
    return None
  response_dict = json.loads(response.text)
  return response_dict['results']

In [7]:
TMDB_AUTH = userdata.get("TMDB_AUTHORIZATION")

movies = []
page_max = 2501 # not inclusive
for page in range(1, page_max):
  results = None
  try:
    results = get_popular_page(page, TMDB_AUTH)
  except Exception as e:
    print(f"Broke at {page} with error {e}")
    page_max = page
    break
  if results:
    movies.extend(results)

json_path = f'/content/drive/My Drive/Colab_Notebooks/popular_tmdb_endpoint_until_page_{page_max}.json'

with open(json_path, 'w') as f:
    json.dump(movies, f)


# # Load the movies list from the JSON file
# with open(json_path, 'r') as f:
#     movies = json.load(f)

# pp = pprint.PrettyPrinter(indent=4)
# #print(len(movies))
# #pp.pprint(movies)
# print(len(movies))
# print(json.dumps(movies))

# https://image.tmdb.org/t/p/original/[poster_path]

# https://image.tmdb.org/t/p/original
# https://image.tmdb.org/t/p/original/vKVUsumbCzK5Kn3aDpKM4EizKCA.jpg
# https://image.tmdb.org/t/p/original/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg

Broke at 501 with error 'results'


In [8]:
json_path = f'/content/drive/My Drive/Colab_Notebooks/popular_tmdb_endpoint_until_page_501.json'

# Load the movies list from the JSON file
with open(json_path, 'r') as f:
    movies = json.load(f)

for i, m in enumerate(movies):
  print(m)
  if i == 5:
    break


{'adult': False, 'backdrop_path': '/yDHYTfA3R0jFYba16jBB1ef8oIt.jpg', 'genre_ids': [28, 35, 878], 'id': 533535, 'original_language': 'en', 'original_title': 'Deadpool & Wolverine', 'overview': 'A listless Wade Wilson toils away in civilian life with his days as the morally flexible mercenary, Deadpool, behind him. But when his homeworld faces an existential threat, Wade must reluctantly suit-up again with an even more reluctant Wolverine.', 'popularity': 7253.796, 'poster_path': '/8cdWjvZQUExUUTzyp4t6EDMubfO.jpg', 'release_date': '2024-07-24', 'title': 'Deadpool & Wolverine', 'video': False, 'vote_average': 7.779, 'vote_count': 2137}
{'adult': False, 'backdrop_path': '/stKGOm8UyhuLPR9sZLjs5AkmncA.jpg', 'genre_ids': [16, 10751, 12, 35], 'id': 1022789, 'original_language': 'en', 'original_title': 'Inside Out 2', 'overview': "Teenager Riley's mind headquarters is undergoing a sudden demolition to make room for something entirely unexpected: new Emotions! Joy, Sadness, Anger, Fear and Disg

CLIP Processing on each popular movie

In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests
import json
from io import BytesIO
import torch

# Load the model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
NAMESPACE = "clip-vit-large-patch14"

index = pc.Index("movie-posters-v2-clip")

json_path = f'/content/drive/My Drive/Colab_Notebooks/popular_tmdb_endpoint_until_page_501.json'

# Load the movies list from the JSON file
with open(json_path, 'r') as f:
    movies = json.load(f)

def get_image_embedding(poster_url):
    # Fetch the image from the URL
    try:
      poster_url = "https://image.tmdb.org/t/p/original" + poster_url
      response = requests.get(poster_url)
      image = Image.open(BytesIO(response.content)).convert('RGB')

      # Preprocess the image
      inputs = processor(images=image, return_tensors="pt")

      # Generate image embedding
      with torch.no_grad():
          outputs = model.get_image_features(**inputs)
          embedding = outputs.squeeze().numpy()

      return embedding
    except Exception as e:
      print(f"Failed to get embedding for {poster_url}: {e}")
      return None

def prepare_metadata(movie):
    return {
        "adult": movie.get("adult", False),
        "backdrop_path": movie.get("backdrop_path", ""),
        "genre_ids": ','.join(map(str, movie.get("genre_ids", []))),
        "id": str(movie.get("id", "")),
        "original_language": movie.get("original_language", ""),
        "original_title": movie.get("original_title", ""),
        "overview": movie.get("overview", ""),
        "popularity": movie.get("popularity", 0.0),
        "poster_path": movie.get("poster_path", ""),
        "release_date": movie.get("release_date", ""),
        "title": movie.get("title", ""),
        "video": movie.get("video", False),
        "vote_average": movie.get("vote_average", 0.0),
        "vote_count": movie.get("vote_count", 0)
    }

def process_and_upload():
    for movie in movies:
        imdb_id = str(movie.get('id', '')) # Ensure ID is a string
        poster_url = movie.get('poster_path', '')
        print(f"Processing {imdb_id} - {movie.get('title')}")

        # Generate image embedding
        try:
            embedding = get_image_embedding(poster_url)
        except Exception as e:
            print(f"Failed to get embedding for {poster_url}: {e}")
            continue

        metadata = prepare_metadata(movie)

        try:
        # Upload to Pinecone with the specified namespace and metadata as a dictionary
          index.upsert(
              vectors=[(imdb_id, embedding.tolist(), metadata)],
              namespace=NAMESPACE
          )
        except Exception as e:
            print(f"Failed to upload to Pinecone: {e}")

process_and_upload()

Processing 533535 - Deadpool & Wolverine
Processing 1022789 - Inside Out 2
Processing 718821 - Twisters
Processing 519182 - Despicable Me 4
Processing 573435 - Bad Boys: Ride or Die
Processing 1160018 - Kill
Processing 945961 - Alien: Romulus
Processing 1079091 - It Ends with Us
Processing 762441 - A Quiet Place: Day One
Processing 831815 - Saving Bikini Bottom: The Sandy Cheeks Movie
Processing 588648 - Dragonkeeper
Processing 959092 - Mayhem!
Processing 1129598 - Prey
Processing 1281826 - One Fast Move
Processing 748783 - The Garfield Movie
Processing 1059064 - The Instigators
Processing 653346 - Kingdom of the Planet of the Apes
Processing 786892 - Furiosa: A Mad Max Saga
Processing 1066262 - The Convert
Processing 1002088 - Mission: Cross
Processing 1094138 - Jackpot!
Processing 956842 - Fly Me to the Moon
Processing 1311550 - House of Ga'a
Processing 1166073 - Breaking and Re-entering
Processing 1064375 - Paradox Effect
Processing 1225377 - The Mouse Trap
Processing 974262 - Desce

In [ ]:
print("done")